<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin: 10px 0; color: #000;">
    <center> <strong>A project:</strong> This is a study of new methods for me. Created by <strong> Julliane Reyes </strong> for the completion of requirements in DS150. </center>
</div>

 # <strong> Phase 1: Library Installs and Imports <strong>
 ***

In [49]:
#%pip install yfinance

In [50]:
import yfinance as yf
#---------------------------------------------------
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [51]:
''' 
A .py file containing the machine learning models for the project. 
Just an experimentation to be honest.
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

from Models.RY_MLEvaluation import evaluate_models, best_model

In [52]:
# Required Libraries ---------------------------
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model #GARCH
# For LSTM ------------------------------------
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
# Misc. ---------------------------------------
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

 # <strong> Phase 2: Stock Vars. Preprocessing
 ***

In [77]:
#Download all stock data

allstocks = ["TSLA", "GME", "AMD", "AAPL", "SPY"]
allstocksdata = yf.download(allstocks, start="2020-01-01", end="2025-03-17")

[*********************100%***********************]  5 of 5 completed


In [78]:
allstocksdata.head(5)

Price           Close                                                 High  \
Ticker           AAPL        AMD     GME         SPY       TSLA       AAPL   
Date                                                                         
2020-01-02  72.716072  49.099998  1.5775  301.194977  28.684000  72.776598   
2020-01-03  72.009125  48.599998  1.4700  298.914154  29.534000  72.771752   
2020-01-06  72.582916  48.389999  1.4625  300.054535  30.102667  72.621654   
2020-01-07  72.241554  48.250000  1.3800  299.210876  31.270666  72.849231   
2020-01-08  73.403648  47.830002  1.4300  300.805511  32.809334  73.706279   

Price                                                 ...       Open  \
Ticker            AMD     GME         SPY       TSLA  ...       AAPL   
Date                                                  ...              
2020-01-02  49.250000  1.6175  301.213537  28.713333  ...  71.721019   
2020-01-03  49.389999  1.5625  300.054527  30.266666  ...  71.941336   
2020-01-06  48.860001  1.4775  300.137973  30.104000  ...  71.127873   
2020-01-07  49.389999  1.4575  299.961845  31.441999  ...  72.592601   
2020-01-08  48.299999  1.4625  302.038574  33.232666  ...  71.943759   

Price                                                    Volume            \
Ticker            AMD     GME         SPY       TSLA       AAPL       AMD   
Date                                                                        
2020-01-02  46.860001  1.5350  299.961913  28.299999  135480400  80331100   
2020-01-03  48.029999  1.5525  297.755249  29.366667  146322800  73127400   
2020-01-06  48.020000  1.4500  297.134071  29.364668  118387200  47934900   
2020-01-07  49.349998  1.4425  299.479722  30.760000  108872000  58061400   
2020-01-08  47.849998  1.3725  299.405545  31.580000  132079200  53767000   

Price                                      
Ticker           GME       SPY       TSLA  
Date                                       
2020-01-02  17814400  59151200  142981500  
2020-01-03  14175600  77709700  266677500  
2020-01-06  13579200  55653900  151995000  
2020-01-07  20912000  40496400  268231500  
2020-01-08  22517600  68296000  467164500  

[5 rows x 25 columns]

In [80]:
''' 
This script selects numerical columns from the dataset `allstocksdata`,
standardizes their values using `StandardScaler`, and updates the dataset
with the transformed numerical data. Standardization ensures the features
have a mean of 0 and a standard deviation of 1 for improved model performance.
'''
allstocksdata.ffill(inplace=True)
numerical_cols = allstocksdata.select_dtypes(include=['float64', 'int64']).columns.tolist()
scaler = StandardScaler()
allstocksdata[numerical_cols] = scaler.fit_transform(allstocksdata[numerical_cols])

#---------------------------------------------------

''' 
Seperating th data into individual dataframes for each stock after collectiv preprocessing.
This is done to make it easier to work with the data for each stock.
'''

stock_dataframes = {}
for stock in allstocks:
    stock_dataframes[stock] = allstocksdata.xs(stock, level=1, axis=1)

TSLA_data = stock_dataframes["TSLA"] # NASDAQ
AMD_data = stock_dataframes["AMD"]   # NASDAQ
AAPL_data = stock_dataframes["AAPL"] # NASDAQ
GME_data = stock_dataframes["GME"]   # NYSE
SPY_data = stock_dataframes["SPY"]   # ARCX

 ## _Phase 2.1: Tesla, Incorporated (NASDAQ: TSLA)_

In [65]:
TSLA_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-2.190752,-2.195758,-2.187560,-2.190051,0.232423
2020-01-03,-2.180945,-2.178259,-2.175587,-2.177783,1.759327
2020-01-06,-2.174384,-2.180092,-2.173163,-2.177806,0.343685
2020-01-07,-2.160909,-2.165018,-2.162646,-2.161759,1.778510
2020-01-08,-2.143157,-2.144846,-2.150940,-2.152328,4.234141


In [66]:
TSLA_data.info() #shows that date is already the index

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1307 entries, 2020-01-02 to 2025-03-14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1307 non-null   float64
 1   High    1307 non-null   float64
 2   Low     1307 non-null   float64
 3   Open    1307 non-null   float64
 4   Volume  1307 non-null   float64
dtypes: float64(5)
memory usage: 61.3 KB


 ## _Phase 2.2: GameStop Corporation (NYSE: GME)_

In [81]:
GME_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.407481,-1.370269,-1.426257,-1.396096,-0.091713
2020-01-03,-1.414576,-1.373623,-1.430276,-1.394957,-0.154670
2020-01-06,-1.415071,-1.378805,-1.434471,-1.401631,-0.164989
2020-01-07,-1.420516,-1.380025,-1.437267,-1.402119,-0.038119
2020-01-08,-1.417216,-1.379720,-1.437791,-1.406677,-0.010339


 ## _Phase 2.3: Advanced Micro Devices, Incorporated (NASDAQ: AMD)_

In [68]:
AMD_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.550585,-1.573854,-1.590573,-1.610042,0.605469
2020-01-03,-1.564716,-1.569975,-1.564377,-1.577098,0.355711
2020-01-06,-1.570651,-1.584663,-1.555165,-1.577379,-0.517736
2020-01-07,-1.574607,-1.569975,-1.549984,-1.539929,-0.166641
2020-01-08,-1.586477,-1.600183,-1.575892,-1.582166,-0.315532


 ## _Phase 2.4: Apple, Incorporated (NASDAQ: AAPL)_

In [69]:
AAPL_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.869401,-1.894480,-1.870233,-1.890069,0.876884
2020-01-03,-1.885411,-1.894589,-1.863004,-1.885076,1.082290
2020-01-06,-1.872417,-1.897972,-1.883700,-1.903511,0.553059
2020-01-07,-1.880147,-1.892843,-1.857595,-1.870316,0.372797
2020-01-08,-1.853830,-1.873529,-1.859361,-1.885021,0.812449


 ## _Phase 2.5: SPDR S&P 500 ETF Trust (ARCX: SPY)_

In [70]:
SPY_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.336456,-1.366067,-1.328759,-1.348854,-0.520699
2020-01-03,-1.362625,-1.379383,-1.343961,-1.374150,-0.071035
2020-01-06,-1.349540,-1.378424,-1.351828,-1.381270,-0.605437
2020-01-07,-1.359220,-1.380448,-1.331842,-1.354381,-0.972696
2020-01-08,-1.340924,-1.356588,-1.327271,-1.355231,-0.299125


 # <strong> Phase 3: Stock Vars. Prediction Modeling
 ***

 ## _Phase 3.1: Tesla, Incorporated (NASDAQ: TSLA)_

In [71]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#--------------------------------------------------- DATA SET-UPS

train_data = TSLA_data['Close'][:int(len(TSLA_data)*0.8)] #**
test_data = TSLA_data['Close'][int(len(TSLA_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#--------------------------------------------------- LINEAR REGRESSION

LinearReg_model.fit(X_train, y_train)
TSLA_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#--------------------------------------------------- AUTOREGRESSIVE MODEL

import warnings # cant fix the valuewarnin rn, will fix later. suppress now.
warnings.filterwarnings("ignore")

close_series = train_data['Close']
AuReg_model.fit(close_series, lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=10) #*

#--------------------------------------------------- MOVING AVERAGE

MovAve_model.fit(train_data['Close'], window_size=5)
TSLA_MovAve_pred = MovAve_model.predict() #*
TSLA_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred) #**

#--------------------------------------------------- ARIMA

train_data_values = train_data['Close']
ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#--------------------------------------------------- SARIMA

train_data_values = train_data['Close']
SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#--------------------------------------------------- GARCH

GARCH_model.fit(train_data['Close'])
TSLA_GARCH_pred = GARCH_model.predict(steps=10) #*
#TSLA_GARCH_pred = TSLA_GARCH_pred.mean.values[-1] #**

#--------------------------------------------------- LSTM

train_data, test_data = train_test_split(TSLA_data['Close'], test_size=0.2, shuffle=False) #*

def create_dataset(data, time_steps=1, n_steps_ahead=1):
    X, y = [], []
    for i in range(len(data) - time_steps - n_steps_ahead + 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[(i + time_steps):(i + time_steps + n_steps_ahead), 0])
    return np.array(X), np.array(y)

train_data_values = train_data.values.reshape(-1, 1)
X_train, y_train = create_dataset(train_data_values, time_steps=1, n_steps_ahead=10) 

test_data_values = test_data.values.reshape(-1, 1)
X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=10) 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train_LSTM.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*
TSLA_LSTM_pred = TSLA_LSTM_pred[:10] #**
TSLA_LSTM_pred = pd.DataFrame(TSLA_LSTM_pred, columns=['LSTM_Predicted_Close']) #**

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [72]:
TSLA_LR_pred = pd.DataFrame(TSLA_LR_pred, columns=['LR_Predicted_Close'])
TSLA_AR_pred = pd.DataFrame(TSLA_AR_pred, columns=['AR_Predicted_Close'])
TSLA_MovAve_pred = pd.DataFrame(TSLA_MovAve_pred, columns=['MovAve_Predicted_Close'])
TSLA_ARIMA_pred = pd.DataFrame(TSLA_ARIMA_pred).rename(columns={'predicted_mean': 'ARIMA_Predicted_Close'})
TSLA_SARIMA_pred = pd.DataFrame(TSLA_SARIMA_pred).rename(columns={'predicted_mean': 'SARIMA_Predicted_Close'})
TSLA_GARCH_pred = pd.DataFrame(TSLA_GARCH_pred, columns=['GARCH_Predicted_Close'])

TSLA_LR_pred.reset_index(drop=True, inplace=True)
TSLA_AR_pred.reset_index(drop=True, inplace=True)
TSLA_MovAve_pred.reset_index(drop=True, inplace=True)
TSLA_ARIMA_pred.reset_index(drop=True, inplace=True)
TSLA_SARIMA_pred.reset_index(drop=True, inplace=True)
TSLA_GARCH_pred.reset_index(drop=True, inplace=True)
TSLA_LSTM_pred.reset_index(drop=True, inplace=True)

TSLA_all_pred = pd.concat([TSLA_LR_pred, 
                           TSLA_AR_pred, 
                           TSLA_MovAve_pred, 
                           TSLA_ARIMA_pred, 
                           TSLA_SARIMA_pred, 
                           TSLA_GARCH_pred, 
                           TSLA_LSTM_pred
                           ], 
                           axis=1)
TSLA_all_pred.reset_index(drop=True, inplace=True)

last_date = test_data.index[-1]
new_index = pd.date_range(start=last_date, periods=10, freq='B')  # 'B' for business days
TSLA_all_pred.index = new_index

TSLA_all_pred

,LR_Predicted_Close,AR_Predicted_Close,MovAve_Predicted_Close,ARIMA_Predicted_Close,SARIMA_Predicted_Close,GARCH_Predicted_Close,LSTM_Predicted_Close
2025-03-14,-0.188259,-0.213745,-0.252844,-0.215395,-0.195224,0.100914,-0.169580
2025-03-17,-0.190092,-0.211085,-0.248492,-0.216567,-0.207323,0.100914,-0.171510
2025-03-18,-0.181387,-0.208420,-0.249362,-0.215875,-0.216610,0.100914,-0.162343
2025-03-19,-0.347464,-0.205786,-0.237853,-0.216283,-0.215896,0.100914,-0.338168
2025-03-20,-0.432220,-0.203171,-0.241186,-0.216042,-0.195732,0.100914,-0.427168
2025-03-21,-0.480325,-0.200574,-0.245948,-0.216185,-0.196595,0.100914,-0.476936
2025-03-24,-0.456158,-0.197996,-0.244568,-0.216101,-0.195781,0.100914,-0.451904
2025-03-25,-0.494069,-0.195436,-0.243783,-0.216150,-0.186920,0.100914,-0.491198
2025-03-26,-0.466237,-0.192895,-0.242668,-0.216121,-0.189877,0.100914,-0.462336
2025-03-27,-0.468871,-0.190372,-0.243631,-0.216138,-0.191668,0.100914,-0.465065


### <strong> <center> Evaluation </center>
***

In [92]:
from Models.RY_MLEvaluation import evaluate_models, best_model

#--------------------------------------------------- RERUNS

TSLA_LR_predRR = LinearReg_model.predict(X_test)

AuReg_model.fit(close_series, lags=3)
TSLA_AR_predRR = AuReg_model.predict(steps=len(y_test))

window_size = 5
current_window = train_data.iloc[-window_size:].tolist()
TSLA_MovAve_predRR = []

for _ in range(len(y_test)):
    next_pred = np.mean(current_window)
    TSLA_MovAve_predRR.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_predRR = np.array(TSLA_MovAve_predRR)

ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_predRR = ARIMA_model.predict(steps=len(y_test))

SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_predRR = SARIMA_model.predict(steps=len(y_test))
                                        
GARCH_model.fit(train_data)
TSLA_GARCH_predRR = GARCH_model.predict(steps=len(y_test))

X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=len(y_test))

X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
TSLA_LSTM_predRR = LSTM_model.predict(X_test_LSTM)

#--------------------------------------------------- SHAPE CHECK (prior to eval.)
predictions = {
    'Linear Regression': TSLA_LR_predRR.flatten(),
    'Autoregressive': TSLA_AR_predRR,
    'Moving Average': TSLA_MovAve_predRR,
    'ARIMA': TSLA_ARIMA_predRR,
    'SARIMA': TSLA_SARIMA_predRR,
    'GARCH': TSLA_GARCH_predRR,
    'LSTM': TSLA_LSTM_predRR.flatten()
}

print("Shapes of Predictions:")
for model_name, preds in predictions.items():
    print(f"{model_name}: {preds.shape}")
print(f"y_test shape: {y_test.shape}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Shapes of Predictions:
Linear Regression: (252,)
Autoregressive: (252,)
Moving Average: (252,)
ARIMA: (252,)
SARIMA: (252,)
GARCH: (252,)
LSTM: (10,)
y_test shape: (10, 252)


In [ ]:
y_test = pd.DataFrame(y_test) #why does this have 252 columns?
y_test

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,0.963400,1.017435,1.011205,0.953205,0.982318,1.039639,1.004634,0.999650,1.061955,1.052779,...,2.223698,2.240220,2.210963,2.091180,2.059858,2.025781,2.029223,1.920340,2.025093,1.905654
1,1.017435,1.011205,0.953205,0.982318,1.039639,1.004634,0.999650,1.061955,1.052779,1.041224,...,2.240220,2.210963,2.091180,2.059858,2.025781,2.029223,1.920340,2.025093,1.905654,1.826373
2,1.011205,0.953205,0.982318,1.039639,1.004634,0.999650,1.061955,1.052779,1.041224,1.001169,...,2.210963,2.091180,2.059858,2.025781,2.029223,1.920340,2.025093,1.905654,1.826373,1.897508
3,0.953205,0.982318,1.039639,1.004634,0.999650,1.061955,1.052779,1.041224,1.001169,1.035600,...,2.091180,2.059858,2.025781,2.029223,1.920340,2.025093,1.905654,1.826373,1.897508,1.778758
4,0.982318,1.039639,1.004634,0.999650,1.061955,1.052779,1.041224,1.001169,1.035600,1.067986,...,2.059858,2.025781,2.029223,1.920340,2.025093,1.905654,1.826373,1.897508,1.778758,1.815587
5,1.039639,1.004634,0.999650,1.061955,1.052779,1.041224,1.001169,1.035600,1.067986,1.122188,...,2.025781,2.029223,1.920340,2.025093,1.905654,1.826373,1.897508,1.778758,1.815587,1.639585
6,1.004634,0.999650,1.061955,1.052779,1.041224,1.001169,1.035600,1.067986,1.122188,1.141733,...,2.029223,1.920340,2.025093,1.905654,1.826373,1.897508,1.778758,1.815587,1.639585,1.586119
7,0.999650,1.061955,1.052779,1.041224,1.001169,1.035600,1.067986,1.122188,1.141733,1.130483,...,1.920340,2.025093,1.905654,1.826373,1.897508,1.778758,1.815587,1.639585,1.586119,1.619965
8,1.061955,1.052779,1.041224,1.001169,1.035600,1.067986,1.122188,1.141733,1.130483,1.114120,...,2.025093,1.905654,1.826373,1.897508,1.778758,1.815587,1.639585,1.586119,1.619965,1.534488
9,1.052779,1.041224,1.001169,1.035600,1.067986,1.122188,1.141733,1.130483,1.114120,1.103211,...,1.905654,1.826373,1.897508,1.778758,1.815587,1.639585,1.586119,1.619965,1.534488,1.665171


In [ ]:
TSLA_LSTM_predRR = pd.DataFrame(TSLA_LSTM_predRR) #LSTM still produces just 10. fixxxxxxx
TSLA_LSTM_predRR

,0
0,0.848077
1,0.860875
2,0.894561
3,0.890649
4,0.854542
5,0.872610
6,0.908566
7,0.886531
8,0.883413
9,0.922740


 ## _Phase 3.2: GameStop Corporation (NYSE: GME)_

In [82]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#--------------------------------------------------- DATA SET-UPS

train_data = GME_data['Close'][:int(len(GME_data)*0.8)] #**
test_data = GME_data['Close'][int(len(GME_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#--------------------------------------------------- LINEAR REGRESSION

LinearReg_model.fit(X_train, y_train)
GME_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#--------------------------------------------------- AUTOREGRESSIVE MODEL

import warnings # cant fix the valuewarnin rn, will fix later. suppress now.
warnings.filterwarnings("ignore")

close_series = train_data['Close']
AuReg_model.fit(close_series, lags=3)
GME_AR_pred = AuReg_model.predict(steps=10) #*

#--------------------------------------------------- MOVING AVERAGE

MovAve_model.fit(train_data['Close'], window_size=5)
GME_MovAve_pred = MovAve_model.predict() #*
GME_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    GME_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

GME_MovAve_pred = np.array(GME_MovAve_pred) #**

#--------------------------------------------------- ARIMA

train_data_values = train_data['Close']
ARIMA_model.fit(train_data_values, order=(1, 1, 1))
GME_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#--------------------------------------------------- SARIMA

train_data_values = train_data['Close']
SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
GME_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#--------------------------------------------------- GARCH

GARCH_model.fit(train_data['Close'])
GME_GARCH_pred = GARCH_model.predict(steps=10) #*
#GME_GARCH_pred = GME_GARCH_pred.mean.values[-1] #**

#--------------------------------------------------- LSTM

train_data, test_data = train_test_split(GME_data['Close'], test_size=0.2, shuffle=False) #*

def create_dataset(data, time_steps=1, n_steps_ahead=1):
    X, y = [], []
    for i in range(len(data) - time_steps - n_steps_ahead + 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[(i + time_steps):(i + time_steps + n_steps_ahead), 0])
    return np.array(X), np.array(y)

train_data_values = train_data.values.reshape(-1, 1)
X_train, y_train = create_dataset(train_data_values, time_steps=1, n_steps_ahead=10) 

test_data_values = test_data.values.reshape(-1, 1)
X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=10) 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train_LSTM.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
GME_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*
GME_LSTM_pred = GME_LSTM_pred[:10] #**
GME_LSTM_pred = pd.DataFrame(GME_LSTM_pred, columns=['LSTM_Predicted_Close']) #**

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [83]:
GME_LR_pred = pd.DataFrame(GME_LR_pred, columns=['LR_Predicted_Close']) #**
GME_AR_pred = pd.DataFrame(GME_AR_pred, columns=['AR_Predicted_Close']) #**
GME_MovAve_pred = pd.DataFrame(GME_MovAve_pred, columns=['MovAve_Predicted_Close']) #**
GME_ARIMA_pred = pd.DataFrame(GME_ARIMA_pred).rename(columns={'predicted_mean': 'ARIMA_Predicted_Close'}) #***
GME_SARIMA_pred = pd.DataFrame(GME_SARIMA_pred).rename(columns={'predicted_mean': 'SARIMA_Predicted_Close'}) #***
GME_GARCH_pred = pd.DataFrame(GME_GARCH_pred, columns=['GARCH_Predicted_Close']) #***

GME_LR_pred.reset_index(drop=True, inplace=True) #*
GME_AR_pred.reset_index(drop=True, inplace=True) #*
GME_MovAve_pred.reset_index(drop=True, inplace=True) #*
GME_ARIMA_pred.reset_index(drop=True, inplace=True) #*
GME_SARIMA_pred.reset_index(drop=True, inplace=True) #*
GME_GARCH_pred.reset_index(drop=True, inplace=True) #*
GME_LSTM_pred.reset_index(drop=True, inplace=True) #*

GME_all_pred = pd.concat([GME_LR_pred, #**
                           GME_AR_pred, #*
                           GME_MovAve_pred, #* 
                           GME_ARIMA_pred, #*
                           GME_SARIMA_pred, #*
                           GME_GARCH_pred, #*
                           GME_LSTM_pred #*
                           ], 
                           axis=1)
GME_all_pred.reset_index(drop=True, inplace=True) #*
last_date = test_data.index[-1]
new_index = pd.date_range(start=last_date, periods=10, freq='B')  # 'B' for business days
GME_all_pred.index = new_index

GME_all_pred #*

,LR_Predicted_Close,AR_Predicted_Close,MovAve_Predicted_Close,ARIMA_Predicted_Close,SARIMA_Predicted_Close,GARCH_Predicted_Close,LSTM_Predicted_Close
2025-03-14,-0.559361,-0.567480,-0.611732,-0.580246,-0.594345,-0.137462,-0.548909
2025-03-17,-0.557421,-0.554539,-0.608775,-0.578144,-0.614288,-0.137462,-0.547125
2025-03-18,-0.513446,-0.545254,-0.604567,-0.578821,-0.612829,-0.137462,-0.506163
2025-03-19,-0.495985,-0.534977,-0.601497,-0.578603,-0.595563,-0.137462,-0.489157
2025-03-20,-0.510212,-0.525345,-0.600058,-0.578673,-0.541915,-0.137462,-0.503120
2025-03-21,-0.493398,-0.515697,-0.605326,-0.578650,-0.570497,-0.137462,-0.486600
2025-03-24,-0.493398,-0.506275,-0.604045,-0.578658,-0.540262,-0.137462,-0.486600
2025-03-25,-0.532847,-0.496983,-0.603099,-0.578655,-0.548292,-0.137462,-0.524465
2025-03-26,-0.547074,-0.487854,-0.602805,-0.578656,-0.557305,-0.137462,-0.537625
2025-03-27,-0.528320,-0.478871,-0.603066,-0.578656,-0.553398,-0.137462,-0.520188


### <strong> <center> Evaluation </center>
***

In [ ]:
TSLA_LR_pred = LinearReg_model.predict(X_test)

AuReg_model.fit(close_series, lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=len(y_test))

window_size = 5
current_window = train_data.iloc[-window_size:].tolist()
TSLA_MovAve_pred = []

for _ in range(len(y_test)):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred)

ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=len(y_test))

SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=len(y_test))
                                        
GARCH_model.fit(train_data)
TSLA_GARCH_pred = GARCH_model.predict(steps=len(y_test))

X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=len(y_test))

X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM)

#--------------------------------------------------- SHAPE CHECK (prior to eval.)
print("Shapes of Predictions:")
for model_name, preds in predictions.items():
    print(f"{model_name}: {preds.shape}")
print(f"y_test shape: {y_test.shape}")

 ## _Phase 3.3: Advanced Micro Devices, Incorporated (NASDAQ: AMD)_

In [84]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#--------------------------------------------------- DATA SET-UPS

train_data = AMD_data['Close'][:int(len(AMD_data)*0.8)] #**
test_data = AMD_data['Close'][int(len(AMD_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#--------------------------------------------------- LINEAR REGRESSION

LinearReg_model.fit(X_train, y_train)
AMD_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#--------------------------------------------------- AUTOREGRESSIVE MODEL

import warnings # cant fix the valuewarnin rn, will fix later. suppress now.
warnings.filterwarnings("ignore")

close_series = train_data['Close']
AuReg_model.fit(close_series, lags=3)
AMD_AR_pred = AuReg_model.predict(steps=10) #*

#--------------------------------------------------- MOVING AVERAGE

MovAve_model.fit(train_data['Close'], window_size=5)
AMD_MovAve_pred = MovAve_model.predict() #*
AMD_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    AMD_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

AMD_MovAve_pred = np.array(AMD_MovAve_pred) #**

#--------------------------------------------------- ARIMA

train_data_values = train_data['Close']
ARIMA_model.fit(train_data_values, order=(1, 1, 1))
AMD_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#--------------------------------------------------- SARIMA

train_data_values = train_data['Close']
SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
AMD_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#--------------------------------------------------- GARCH

GARCH_model.fit(train_data['Close'])
AMD_GARCH_pred = GARCH_model.predict(steps=10) #*
#AMD_GARCH_pred = AMD_GARCH_pred.mean.values[-1] #**

#--------------------------------------------------- LSTM

train_data, test_data = train_test_split(AMD_data['Close'], test_size=0.2, shuffle=False) #*

def create_dataset(data, time_steps=1, n_steps_ahead=1):
    X, y = [], []
    for i in range(len(data) - time_steps - n_steps_ahead + 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[(i + time_steps):(i + time_steps + n_steps_ahead), 0])
    return np.array(X), np.array(y)

train_data_values = train_data.values.reshape(-1, 1)
X_train, y_train = create_dataset(train_data_values, time_steps=1, n_steps_ahead=10) 

test_data_values = test_data.values.reshape(-1, 1)
X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=10) 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train_LSTM.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
AMD_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*
AMD_LSTM_pred = AMD_LSTM_pred[:10] #**
AMD_LSTM_pred = pd.DataFrame(AMD_LSTM_pred, columns=['LSTM_Predicted_Close']) #**

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [85]:
AMD_LR_pred = pd.DataFrame(AMD_LR_pred, columns=['LR_Predicted_Close']) #**
AMD_AR_pred = pd.DataFrame(AMD_AR_pred, columns=['AR_Predicted_Close']) #**
AMD_MovAve_pred = pd.DataFrame(AMD_MovAve_pred, columns=['MovAve_Predicted_Close']) #**
AMD_ARIMA_pred = pd.DataFrame(AMD_ARIMA_pred).rename(columns={'predicted_mean': 'ARIMA_Predicted_Close'}) #***
AMD_SARIMA_pred = pd.DataFrame(AMD_SARIMA_pred).rename(columns={'predicted_mean': 'SARIMA_Predicted_Close'}) #***
AMD_GARCH_pred = pd.DataFrame(AMD_GARCH_pred, columns=['GARCH_Predicted_Close']) #***

AMD_LR_pred.reset_index(drop=True, inplace=True) #*
AMD_AR_pred.reset_index(drop=True, inplace=True) #*
AMD_MovAve_pred.reset_index(drop=True, inplace=True) #*
AMD_ARIMA_pred.reset_index(drop=True, inplace=True) #*
AMD_SARIMA_pred.reset_index(drop=True, inplace=True) #*
AMD_GARCH_pred.reset_index(drop=True, inplace=True) #*
AMD_LSTM_pred.reset_index(drop=True, inplace=True) #*

AMD_all_pred = pd.concat([AMD_LR_pred, #**
                           AMD_AR_pred, #*
                           AMD_MovAve_pred, #* 
                           AMD_ARIMA_pred, #*
                           AMD_SARIMA_pred, #*
                           AMD_GARCH_pred, #*
                           AMD_LSTM_pred #*
                           ], 
                           axis=1)
AMD_all_pred.reset_index(drop=True, inplace=True) #*
last_date = test_data.index[-1]
new_index = pd.date_range(start=last_date, periods=10, freq='B')  # 'B' for business days
AMD_all_pred.index = new_index #*

AMD_all_pred

,LR_Predicted_Close,AR_Predicted_Close,MovAve_Predicted_Close,ARIMA_Predicted_Close,SARIMA_Predicted_Close,GARCH_Predicted_Close,LSTM_Predicted_Close
2025-03-14,2.047267,2.088144,2.016970,2.089661,2.107627,-0.573931,2.037492
2025-03-17,2.497782,2.084456,2.079404,2.088584,2.111940,-0.573931,2.558577
2025-03-18,2.782628,2.082619,2.055016,2.088138,2.100504,-0.573931,2.909598
2025-03-19,2.859264,2.080695,2.055933,2.087953,2.104872,-0.573931,3.006673
2025-03-20,2.852784,2.078722,2.059916,2.087876,2.106775,-0.573931,2.998423
2025-03-21,3.007745,2.076758,2.053448,2.087844,2.096736,-0.573931,3.197786
2025-03-24,3.028876,2.074801,2.060744,2.087831,2.097927,-0.573931,3.225299
2025-03-25,2.916459,2.072847,2.057012,2.087826,2.105122,-0.573931,3.079824
2025-03-26,2.662886,2.070899,2.057411,2.087823,2.124766,-0.573931,2.760095
2025-03-27,2.786009,2.068954,2.057706,2.087822,2.128381,-0.573931,2.913857


### <strong> <center> Evaluation </center>
***

In [ ]:
TSLA_LR_pred = LinearReg_model.predict(X_test)

AuReg_model.fit(close_series, lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=len(y_test))

window_size = 5
current_window = train_data.iloc[-window_size:].tolist()
TSLA_MovAve_pred = []

for _ in range(len(y_test)):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred)

ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=len(y_test))

SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=len(y_test))
                                        
GARCH_model.fit(train_data)
TSLA_GARCH_pred = GARCH_model.predict(steps=len(y_test))

X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=len(y_test))

X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM)

#--------------------------------------------------- SHAPE CHECK (prior to eval.)
print("Shapes of Predictions:")
for model_name, preds in predictions.items():
    print(f"{model_name}: {preds.shape}")
print(f"y_test shape: {y_test.shape}")

 ## _Phase 3.4: Apple, Incorporated (NASDAQ: AAPL)_

In [86]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#--------------------------------------------------- DATA SET-UPS

train_data = AAPL_data['Close'][:int(len(AAPL_data)*0.8)] #**
test_data = AAPL_data['Close'][int(len(AAPL_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#--------------------------------------------------- LINEAR REGRESSION

LinearReg_model.fit(X_train, y_train)
AAPL_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#--------------------------------------------------- AUTOREGRESSIVE MODEL

import warnings # cant fix the valuewarnin rn, will fix later. suppress now.
warnings.filterwarnings("ignore")

close_series = train_data['Close']
AuReg_model.fit(close_series, lags=3)
AAPL_AR_pred = AuReg_model.predict(steps=10) #*

#--------------------------------------------------- MOVING AVERAGE

MovAve_model.fit(train_data['Close'], window_size=5)
AAPL_MovAve_pred = MovAve_model.predict() #*
AAPL_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    AAPL_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

AAPL_MovAve_pred = np.array(AAPL_MovAve_pred) #**

#--------------------------------------------------- ARIMA

train_data_values = train_data['Close']
ARIMA_model.fit(train_data_values, order=(1, 1, 1))
AAPL_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#--------------------------------------------------- SARIMA

train_data_values = train_data['Close']
SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
AAPL_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#--------------------------------------------------- GARCH

GARCH_model.fit(train_data['Close'])
AAPL_GARCH_pred = GARCH_model.predict(steps=10) #*
#AAPL_GARCH_pred = AAPL_GARCH_pred.mean.values[-1] #**

#--------------------------------------------------- LSTM

train_data, test_data = train_test_split(AAPL_data['Close'], test_size=0.2, shuffle=False) #*

def create_dataset(data, time_steps=1, n_steps_ahead=1):
    X, y = [], []
    for i in range(len(data) - time_steps - n_steps_ahead + 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[(i + time_steps):(i + time_steps + n_steps_ahead), 0])
    return np.array(X), np.array(y)

train_data_values = train_data.values.reshape(-1, 1)
X_train, y_train = create_dataset(train_data_values, time_steps=1, n_steps_ahead=10) 

test_data_values = test_data.values.reshape(-1, 1)
X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=10) 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train_LSTM.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
AAPL_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*
AAPL_LSTM_pred = AAPL_LSTM_pred[:10] #**
AAPL_LSTM_pred = pd.DataFrame(AAPL_LSTM_pred, columns=['LSTM_Predicted_Close']) #**

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [87]:
AAPL_LR_pred = pd.DataFrame(AAPL_LR_pred, columns=['LR_Predicted_Close']) #**
AAPL_AR_pred = pd.DataFrame(AAPL_AR_pred, columns=['AR_Predicted_Close']) #**
AAPL_MovAve_pred = pd.DataFrame(AAPL_MovAve_pred, columns=['MovAve_Predicted_Close']) #**
AAPL_ARIMA_pred = pd.DataFrame(AAPL_ARIMA_pred).rename(columns={'predicted_mean': 'ARIMA_Predicted_Close'}) #***
AAPL_SARIMA_pred = pd.DataFrame(AAPL_SARIMA_pred).rename(columns={'predicted_mean': 'SARIMA_Predicted_Close'}) #***
AAPL_GARCH_pred = pd.DataFrame(AAPL_GARCH_pred, columns=['GARCH_Predicted_Close']) #***

AAPL_LR_pred.reset_index(drop=True, inplace=True) #*
AAPL_AR_pred.reset_index(drop=True, inplace=True) #*
AAPL_MovAve_pred.reset_index(drop=True, inplace=True) #*
AAPL_ARIMA_pred.reset_index(drop=True, inplace=True) #*
AAPL_SARIMA_pred.reset_index(drop=True, inplace=True) #*
AAPL_GARCH_pred.reset_index(drop=True, inplace=True) #*
AAPL_LSTM_pred.reset_index(drop=True, inplace=True) #*

AAPL_all_pred = pd.concat([AAPL_LR_pred, #**
                           AAPL_AR_pred, #*
                           AAPL_MovAve_pred, #* 
                           AAPL_ARIMA_pred, #*
                           AAPL_SARIMA_pred, #*
                           AAPL_GARCH_pred, #*
                           AAPL_LSTM_pred #*
                           ], 
                           axis=1)
AAPL_all_pred.reset_index(drop=True, inplace=True) #*

last_date = test_data.index[-1]
new_index = pd.date_range(start=last_date, periods=10, freq='B')  # 'B' for business days
AAPL_all_pred.index = new_index

AAPL_all_pred

,LR_Predicted_Close,AR_Predicted_Close,MovAve_Predicted_Close,ARIMA_Predicted_Close,SARIMA_Predicted_Close,GARCH_Predicted_Close,LSTM_Predicted_Close
2025-03-14,0.571700,0.598620,0.599646,0.600096,0.608873,-0.216319,0.577250
2025-03-17,0.556660,0.596769,0.600908,0.599966,0.610259,-0.216319,0.563863
2025-03-18,0.532191,0.595633,0.593181,0.599890,0.608682,-0.216319,0.542117
2025-03-19,0.429828,0.594482,0.592249,0.599846,0.609820,-0.216319,0.443932
2025-03-20,0.318034,0.593322,0.597261,0.599820,0.620073,-0.216319,0.331445
2025-03-21,0.295586,0.592166,0.596649,0.599805,0.618808,-0.216319,0.308869
2025-03-24,0.292892,0.591016,0.596050,0.599797,0.619494,-0.216319,0.306160
2025-03-25,0.331728,0.589869,0.595078,0.599792,0.623096,-0.216319,0.345217
2025-03-26,0.377074,0.588727,0.595457,0.599789,0.628932,-0.216319,0.390836
2025-03-27,0.387849,0.587589,0.596099,0.599787,0.640191,-0.216319,0.401679


### <strong> <center> Evaluation </center>
***

In [ ]:
TSLA_LR_pred = LinearReg_model.predict(X_test)

AuReg_model.fit(close_series, lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=len(y_test))

window_size = 5
current_window = train_data.iloc[-window_size:].tolist()
TSLA_MovAve_pred = []

for _ in range(len(y_test)):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred)

ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=len(y_test))

SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=len(y_test))
                                        
GARCH_model.fit(train_data)
TSLA_GARCH_pred = GARCH_model.predict(steps=len(y_test))

X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=len(y_test))

X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM)

#--------------------------------------------------- SHAPE CHECK (prior to eval.)
print("Shapes of Predictions:")
for model_name, preds in predictions.items():
    print(f"{model_name}: {preds.shape}")
print(f"y_test shape: {y_test.shape}")

 ## _Phase 3.5: SPDR S&P 500 ETF Trust (ARCX: SPY)_

In [88]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#--------------------------------------------------- DATA SET-UPS

train_data = SPY_data['Close'][:int(len(SPY_data)*0.8)] #**
test_data = SPY_data['Close'][int(len(SPY_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#--------------------------------------------------- LINEAR REGRESSION

LinearReg_model.fit(X_train, y_train)
SPY_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#--------------------------------------------------- AUTOREGRESSIVE MODEL

import warnings # cant fix the valuewarnin rn, will fix later. suppress now.
warnings.filterwarnings("ignore")

close_series = train_data['Close']
AuReg_model.fit(close_series, lags=3)
SPY_AR_pred = AuReg_model.predict(steps=10) #*

#--------------------------------------------------- MOVING AVERAGE

MovAve_model.fit(train_data['Close'], window_size=5)
SPY_MovAve_pred = MovAve_model.predict() #*
SPY_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    SPY_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

SPY_MovAve_pred = np.array(SPY_MovAve_pred) #**

#--------------------------------------------------- ARIMA

train_data_values = train_data['Close']
ARIMA_model.fit(train_data_values, order=(1, 1, 1))
SPY_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#--------------------------------------------------- SARIMA

train_data_values = train_data['Close']
SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
SPY_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#--------------------------------------------------- GARCH

GARCH_model.fit(train_data['Close'])
SPY_GARCH_pred = GARCH_model.predict(steps=10) #*
#SPY_GARCH_pred = SPY_GARCH_pred.mean.values[-1] #**

#--------------------------------------------------- LSTM

train_data, test_data = train_test_split(SPY_data['Close'], test_size=0.2, shuffle=False) #*

def create_dataset(data, time_steps=1, n_steps_ahead=1):
    X, y = [], []
    for i in range(len(data) - time_steps - n_steps_ahead + 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[(i + time_steps):(i + time_steps + n_steps_ahead), 0])
    return np.array(X), np.array(y)

train_data_values = train_data.values.reshape(-1, 1)
X_train, y_train = create_dataset(train_data_values, time_steps=1, n_steps_ahead=10) 

test_data_values = test_data.values.reshape(-1, 1)
X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=10) 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train_LSTM.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
SPY_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*
SPY_LSTM_pred = SPY_LSTM_pred[:10] #**
SPY_LSTM_pred = pd.DataFrame(SPY_LSTM_pred, columns=['LSTM_Predicted_Close']) #**

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [89]:
SPY_LR_pred = pd.DataFrame(SPY_LR_pred, columns=['LR_Predicted_Close']) #**
SPY_AR_pred = pd.DataFrame(SPY_AR_pred, columns=['AR_Predicted_Close']) #**
SPY_MovAve_pred = pd.DataFrame(SPY_MovAve_pred, columns=['MovAve_Predicted_Close']) #**
SPY_ARIMA_pred = pd.DataFrame(SPY_ARIMA_pred).rename(columns={'predicted_mean': 'ARIMA_Predicted_Close'}) #***
SPY_SARIMA_pred = pd.DataFrame(SPY_SARIMA_pred).rename(columns={'predicted_mean': 'SARIMA_Predicted_Close'}) #***
SPY_GARCH_pred = pd.DataFrame(SPY_GARCH_pred, columns=['GARCH_Predicted_Close']) #***

SPY_LR_pred.reset_index(drop=True, inplace=True) #*
SPY_AR_pred.reset_index(drop=True, inplace=True) #*
SPY_MovAve_pred.reset_index(drop=True, inplace=True) #*
SPY_ARIMA_pred.reset_index(drop=True, inplace=True) #*
SPY_SARIMA_pred.reset_index(drop=True, inplace=True) #*
SPY_GARCH_pred.reset_index(drop=True, inplace=True) #*
SPY_LSTM_pred.reset_index(drop=True, inplace=True) #*

SPY_all_pred = pd.concat([SPY_LR_pred, #**
                           SPY_AR_pred, #*
                           SPY_MovAve_pred, #* 
                           SPY_ARIMA_pred, #*
                           SPY_SARIMA_pred, #*
                           SPY_GARCH_pred, #*
                           SPY_LSTM_pred #*
                           ], 
                           axis=1)
SPY_all_pred.reset_index(drop=True, inplace=True) #*

last_date = test_data.index[-1]
new_index = pd.date_range(start=last_date, periods=10, freq='B')  # 'B' for business days
SPY_all_pred.index = new_index

SPY_all_pred

,LR_Predicted_Close,AR_Predicted_Close,MovAve_Predicted_Close,ARIMA_Predicted_Close,SARIMA_Predicted_Close,GARCH_Predicted_Close,LSTM_Predicted_Close
2025-03-14,0.941840,0.947991,0.929597,0.949123,0.954034,-0.195819,0.848077
2025-03-17,0.962408,0.948041,0.947464,0.949594,0.947952,-0.195819,0.860875
2025-03-18,1.016317,0.947336,0.945591,0.949417,0.945676,-0.195819,0.894561
2025-03-19,1.010102,0.946798,0.942550,0.949484,0.947592,-0.195819,0.890649
2025-03-20,0.952237,0.946215,0.943115,0.949459,0.956138,-0.195819,0.854542
2025-03-21,0.981282,0.945644,0.941663,0.949468,0.956646,-0.195819,0.872610
2025-03-24,1.038469,0.945072,0.944076,0.949464,0.959197,-0.195819,0.908566
2025-03-25,1.003546,0.944502,0.943399,0.949466,0.965395,-0.195819,0.886531
2025-03-26,0.998574,0.943932,0.942961,0.949465,0.970598,-0.195819,0.883413
2025-03-27,1.060734,0.943364,0.943043,0.949466,0.981918,-0.195819,0.922740


### <strong> <center> Evaluation </center>
***

In [ ]:
TSLA_LR_pred = LinearReg_model.predict(X_test)

AuReg_model.fit(close_series, lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=len(y_test))

window_size = 5
current_window = train_data.iloc[-window_size:].tolist()
TSLA_MovAve_pred = []

for _ in range(len(y_test)):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred)

ARIMA_model.fit(train_data_values, order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=len(y_test))

SARIMA_model.fit(train_data_values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=len(y_test))
                                        
GARCH_model.fit(train_data)
TSLA_GARCH_pred = GARCH_model.predict(steps=len(y_test))

X_test, y_test = create_dataset(test_data_values, time_steps=1, n_steps_ahead=len(y_test))

X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM)

#--------------------------------------------------- SHAPE CHECK (prior to eval.)
print("Shapes of Predictions:")
for model_name, preds in predictions.items():
    print(f"{model_name}: {preds.shape}")
print(f"y_test shape: {y_test.shape}")

 # <strong> Phase 4: Findings and Summary
 ***